In [1]:
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime

# Load U.S. Airports dataset
airports_data = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", header=None)
airports_data.columns = [
    "Airport ID", "Name", "City", "Country", "IATA", "ICAO",
    "Latitude", "Longitude", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"
]

# Filter for U.S. airports with valid IATA codes
us_airports = airports_data[(airports_data["Country"] == "United States") & airports_data["IATA"].notna()]
us_airports = us_airports[["Name", "City", "IATA", "Latitude", "Longitude"]]

# Prepare list to store direct flight results
direct_flight_results = []

# Set travel date
travel_date = datetime(2024, 11, 25)

# Direct flight calculation function
def calculate_direct_flight(start_airport, end_airport):
    distance = geodesic(
        (start_airport["Latitude"], start_airport["Longitude"]),
        (end_airport["Latitude"], end_airport["Longitude"])
    ).miles
    average_flight_speed = 500  # mph
    fuel_cost_per_mile = 3.5 * 3.0  # gallons per mile * cost per gallon
    travel_time = distance / average_flight_speed
    fuel_cost = distance * fuel_cost_per_mile
    airport_charges = 2500 + 0.1 * distance
    economy_fare = 250 + 0.05 * distance
    first_class_fare = economy_fare * 1.5
    total_revenue = economy_fare * 150  # Assume 150 passengers
    profit = total_revenue - (fuel_cost + airport_charges)

    return {
        "Start Airport": start_airport["City"],
        "End Airport": end_airport["City"],
        "Distance (miles)": distance,
        "Travel Time (hours)": travel_time,
        "Fuel Cost ($)": fuel_cost,
        "Airport Charges ($)": airport_charges,
        "Economy Fare ($)": economy_fare,
        "First Class Fare ($)": first_class_fare,
        "Total Revenue ($)": total_revenue,
        "Profit ($)": profit
    }

# Iterate through all airport combinations for direct flights
for i, start_airport in us_airports.iterrows():
    for j, end_airport in us_airports.iterrows():
        if start_airport["IATA"] != end_airport["IATA"]:  # Avoid same airport combinations
            try:
                flight_data = calculate_direct_flight(start_airport, end_airport)
                direct_flight_results.append(flight_data)
            except Exception as e:
                print(f"Error processing direct flight from {start_airport['City']} to {end_airport['City']}: {e}")

# Save results to CSV
direct_flight_df = pd.DataFrame(direct_flight_results)
direct_flight_df.to_csv("direct_flights.csv", index=False)
print("Direct flight details have been saved to 'direct_flights.csv'.")


Direct flight details have been saved to 'direct_flights.csv'.


In [ ]:
import random
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime

# Load U.S. Airports dataset
airports_data = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", header=None)
airports_data.columns = [
    "Airport ID", "Name", "City", "Country", "IATA", "ICAO",
    "Latitude", "Longitude", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"
]

# Filter for U.S. airports with valid IATA codes
us_airports = airports_data[(airports_data["Country"] == "United States") & airports_data["IATA"].notna()]
us_airports = us_airports[["Name", "City", "IATA", "Latitude", "Longitude"]]

# Define Sunbelt cities (layover hubs)
sunbelt_cities = [
    "Dallas/Fort Worth International Airport",
    "Charlotte Douglas International Airport",
    "Phoenix Sky Harbor International Airport",
    "Miami International Airport"
]

# Filter U.S. airports to include only Sunbelt layover hubs
sunbelt_airports = us_airports[us_airports["Name"].str.contains('|'.join(sunbelt_cities), case=False, na=False)]

# Prepare list to store connecting flight results
connecting_flight_results = []

# Travel date
travel_date = datetime(2024, 11, 25)

# Connecting flight calculation function
def calculate_connecting_flight(start_airport, layover_airport, end_airport):
    # Calculate distances
    try:
        distance_leg1 = geodesic(
            (start_airport["Latitude"], start_airport["Longitude"]),
            (layover_airport["Latitude"], layover_airport["Longitude"])
        ).miles
        distance_leg2 = geodesic(
            (layover_airport["Latitude"], layover_airport["Longitude"]),
            (end_airport["Latitude"], end_airport["Longitude"])
        ).miles
    except ValueError:
        return None  # Skip invalid distances

    total_distance = distance_leg1 + distance_leg2
    average_flight_speed = 500  # mph
    fuel_cost_per_mile = 3.5 * 3.0  # gallons per mile * cost per gallon

    travel_time_leg1 = distance_leg1 / average_flight_speed
    travel_time_leg2 = distance_leg2 / average_flight_speed
    layover_time = random.uniform(2, 5)  # Layover time in hours
    total_travel_time = travel_time_leg1 + travel_time_leg2 + layover_time

    fuel_cost_leg1 = distance_leg1 * fuel_cost_per_mile
    fuel_cost_leg2 = distance_leg2 * fuel_cost_per_mile
    total_fuel_cost = fuel_cost_leg1 + fuel_cost_leg2

    airport_charges_leg1 = 2500 + 0.1 * distance_leg1
    airport_charges_leg2 = 2500 + 0.1 * distance_leg2
    total_airport_charges = airport_charges_leg1 + airport_charges_leg2

    economy_fare = 200 + 0.04 * total_distance  # Lower fare for connecting flights
    first_class_fare = economy_fare * 1.3  # Smaller multiplier for first class
    total_revenue = economy_fare * 150  # Assume 150 passengers
    profit = total_revenue - (total_fuel_cost + total_airport_charges)

    return {
        "Start Airport": start_airport["City"],
        "Layover Airport": layover_airport["City"],
        "End Airport": end_airport["City"],
        "Total Distance (miles)": total_distance,
        "Total Travel Time (hours)": total_travel_time,
        "Fuel Cost ($)": total_fuel_cost,
        "Airport Charges ($)": total_airport_charges,
        "Economy Fare ($)": economy_fare,
        "First Class Fare ($)": first_class_fare,
        "Total Revenue ($)": total_revenue,
        "Profit ($)": profit
    }

# Iterate through all airport combinations for connecting flights
for i, start_airport in us_airports.iterrows():
    for j, layover_airport in sunbelt_airports.iterrows():
        for k, end_airport in us_airports.iterrows():
            if (start_airport["IATA"] != layover_airport["IATA"]
                    and layover_airport["IATA"] != end_airport["IATA"]
                    and start_airport["IATA"] != end_airport["IATA"]):  # Avoid invalid routes
                try:
                    flight_data = calculate_connecting_flight(start_airport, layover_airport, end_airport)
                    if flight_data:
                        connecting_flight_results.append(flight_data)
                except Exception as e:
                    print(f"Error processing connecting flight from {start_airport['City']} via {layover_airport['City']} to {end_airport['City']}: {e}")

# Check if any results were generated
if connecting_flight_results:
    # Save results to CSV
    connecting_flight_df = pd.DataFrame(connecting_flight_results)
    connecting_flight_df.to_csv("connecting_flights.csv", index=False)
    print("Connecting flight details have been saved to 'connecting_flights.csv'.")
else:
    print("No valid connecting flight data was generated.")
